In [1]:
import os
import pandas as pd
import requests

In [2]:
# Download urls from www.tamaraberg.com/street2shop/wheretobuyit/photos.tar
file_path = "C:/Users/heret/Desktop/street2shop/photos.txt"
photos_file = pd.read_table(file_path, header=None)

photos_file = photos_file[0].str.split(pat=",", n=1, expand=True)
photos_file.columns = ["photo", "url"]

print(photos_file.head())
print(photos_file.shape)
print(photos_file["photo"].unique().shape)
print(photos_file["url"].unique().shape)

       photo                                                url
0  000000001  http://s3.amazonaws.com/media.modcloth/images/...
1  000000002  http://s3.amazonaws.com/media.modcloth/images/...
2  000000003  http://s3.amazonaws.com/media.modcloth/images/...
3  000000004  http://s3.amazonaws.com/media.modcloth/images/...
4  000000005  http://media1.modcloth.com/community_outfit_im...
(424840, 2)
(424840,)
(424840,)


In [3]:
# Top url domains
url_domains = photos_file["url"].str.extract(r'(\w*.com)')
print(url_domains[0].value_counts().sort_values(ascending=False))
print(url_domains[0].value_counts().sort_values(ascending=False).sum(), "images") #not all but most included

zappos.com             74353
amazon.com             63982
therealreal.com        55488
neimanmarcus.com       28454
media.com              23819
nordstromimage.com     23434
modcloth.com           18435
targetimg1.com         17098
forever21.com          11390
macys.com               9619
bloomingdales.com       7258
urbanoutfitters.com     6545
scene7.com              6255
kohls.com               3516
anthropologie.com       1942
amazonaws.com           1922
gap.com                 1874
jcrew.com               1664
express.com              476
Name: 0, dtype: int64
357524 images


### Download images + extract broken urls

In [19]:
def image_extraction_3(df):
    #we assume a broken link are those taking >3 secs to load up
    img_path = "C:/Users/heret/Desktop/street2shop/photos/"
    urls = df["url"].tolist()
    photo_ids = df["photo"].tolist()
    broken_urls = pd.DataFrame(columns=["photo", "url"]) 
    print("PHOTOS_TEST DIRECTORY")

    for url, photo_id in zip(urls, photo_ids):
        try:
            r = requests.get(url, timeout=3, 
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36'})
            if r.status_code == requests.codes.ok:
                with open(str(img_path + photo_id.lstrip("0") + ".jpg"), 'wb') as f:
                    f.write(r.content)
        except:
            broken_urls = broken_urls.append({"photo": photo_id, "url": url}, ignore_index=True)
    return broken_urls

In [20]:
start_n = 15532 #beware photos ids start at 1 not at 0 as python indexes. ie. start_n = 5216 will download from 5217
finish_n = 212420 #ie. finish_n = 5218 will download until 5219 included
split_urls = photos_file.loc[start_n:finish_n]
#split_urls = photos_file.sort_index(ascending=False).loc[start_n:finish_n] #starting from end

In [ ]:
%%time
broken_urls = image_extraction_3(split_urls) #started at 19:21

PHOTOS_TEST DIRECTORY


In [11]:
broken_urls.head()

,photo,url
0,000020005,http://g.nordstromimage.com/imagegallery/store...
1,000020010,http://g.nordstromimage.com/imagegallery/store...


In [ ]:
with open("broken_urls.csv", "a") as f: #always appending to same file, careful not to append same output twice
    broken_urls.to_csv(f, index=False)